# 앙상블 (Ensemble) : Voting
- 다양한 모델을 결합하여 예측 성능을 향상시키는 방법
    - hard voting : 여러 개의 예측치에 대해 다수결로 결정
    - soft voting : 여러 개의 예측 확률을 평균내어 결정

In [43]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

#### 위스콘신 유방암 데이터셋 (Wisconsin Breast Cancer Dataset)
유방암의 악성(Malignant)과 양성(Benign)을 분류하기 위해 자주 사용되는 데이터셋
(의학적인 이미지를 바탕으로 유방암 종양의 특징을 수치화한 데이터)
   
**데이터셋 개요**
   - **목적**: 유방암 종양이 악성(Malignant)인지, 양성(Benign)인지 분류
   - **샘플 수**: 569개
   - **특징(Features) 수**: 30개
   - **타겟(Target)**: 0(악성) 또는 1(양성)
  
**데이터 구성**
   1. **Radius mean**: 종양의 평균 반지름
   2. **Texture mean**: 종양의 표면의 거칠기
   3. **Perimeter mean**: 종양의 평균 둘레 길이
   4. **Area mean**: 종양의 평균 면적
   5. **Smoothness mean**: 종양의 매끄러움 정도
   6. **Compactness mean**: 종양의 압축도
   7. **Concavity mean**: 종양의 오목함
   8. **Concave points mean**: 종양의 오목한 점 개수
   9. **Symmetry mean**: 종양의 대칭성
   10. **Fractal dimension mean**: 종양의 프랙탈 차원

In [44]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
# data.DESCR  # 설명

df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [46]:
df.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,1.000000


In [47]:
from sklearn.model_selection import train_test_split

X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

### hard voting


In [48]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

knn_clf = KNeighborsClassifier()
lr_clf = LogisticRegression()
dt_clf = DecisionTreeClassifier(random_state=0)

voting_clf = VotingClassifier(
    estimators=[
        ('knn_clf', knn_clf), 
        ('lr_clf', lr_clf),
        ('dt_clf', dt_clf),
    ], 
    voting='hard'                # voting 방법 선택
)

voting_clf.fit(X_train, y_train)

print('학습 점수:', voting_clf.score(X_train, y_train))
print('평가 점수:', voting_clf.score(X_test, y_test))

학습 점수: 0.9647887323943662
평가 점수: 0.951048951048951


c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [49]:
# hard voting 작동 원리 == 다수결
# 밑에 각 모델 별 예측값에서 다수결을 따른 것을 알 수 있음. 
start, end = 40, 50

voting_pred = voting_clf.predict(X_test[start:end])
print(f'앙상블 예측값: {voting_pred}')

for classifier in [knn_clf, lr_clf, dt_clf] : 
    # 개별 학습 및 예측
    classifier.fit(X_train, y_train)
    pred = classifier.predict(X_test[start:end])
    score = classifier.score(X_test, y_test)

    class_name = classifier.__class__.__name__    # 언더바 두개 
    print(f'{class_name} 개별 정확도 : {score}')
    print(f'{class_name} 예측값: {pred}')

앙상블 예측값: [0 1 0 1 0 0 1 1 1 0]
KNeighborsClassifier 개별 정확도 : 0.9370629370629371
KNeighborsClassifier 예측값: [0 1 0 1 0 0 1 1 1 0]
LogisticRegression 개별 정확도 : 0.9440559440559441
LogisticRegression 예측값: [0 1 0 1 0 0 1 1 1 0]
DecisionTreeClassifier 개별 정확도 : 0.8811188811188811
DecisionTreeClassifier 예측값: [1 1 0 1 0 0 1 1 1 0]


c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### soft voting

In [50]:
voting_clf = VotingClassifier(
    estimators=[
        ('knn_clf', knn_clf), 
        ('lr_clf', lr_clf),
        ('dt_clf', dt_clf),
    ], 
    voting='soft'                # voting 방법 선택
)

voting_clf.fit(X_train, y_train)

print('학습 점수:', voting_clf.score(X_train, y_train))
print('평가 점수:', voting_clf.score(X_test, y_test))    # 전체 평가 지표가 올라감. 

학습 점수: 0.9859154929577465
평가 점수: 0.9370629370629371


c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [51]:
# soft voting 작동 원리 == 각 예측기의 확률값 평균  
start, end = 40, 50

voting_pred = voting_clf.predict_proba(X_test[start:end])
print(f'앙상블 예측 확률: {voting_pred}')

averages = np.full_like(voting_pred, 0)

for classifier in [knn_clf, lr_clf, dt_clf] : 
    # 개별 학습 및 예측
    classifier.fit(X_train, y_train)
    pred = classifier.predict_proba(X_test[start:end])
    score = classifier.score(X_test, y_test)

    averages += pred

    class_name = classifier.__class__.__name__    # 언더바 두개 
    print(f'{class_name} 개별 정확도 : {score}')
    print(f'{class_name} 예측 확률: {pred}')

print('각 모델별 예측 확률의 평균:', averages / 3 )
print(np.array_equal(voting_pred, averages / 3))

앙상블 예측 확률: [[5.70263157e-01 4.29736843e-01]
 [1.08113730e-03 9.98918863e-01]
 [9.99622506e-01 3.77494355e-04]
 [3.35757426e-04 9.99664243e-01]
 [9.00993416e-01 9.90065841e-02]
 [1.00000000e+00 1.75163138e-13]
 [7.79971341e-05 9.99922003e-01]
 [1.83004552e-02 9.81699545e-01]
 [1.14568790e-03 9.98854312e-01]
 [9.32982089e-01 6.70179112e-02]]
KNeighborsClassifier 개별 정확도 : 0.9370629370629371
KNeighborsClassifier 예측 확률: [[0.8 0.2]
 [0.  1. ]
 [1.  0. ]
 [0.  1. ]
 [0.8 0.2]
 [1.  0. ]
 [0.  1. ]
 [0.  1. ]
 [0.  1. ]
 [0.8 0.2]]
LogisticRegression 개별 정확도 : 0.9440559440559441
LogisticRegression 예측 확률: [[9.10789471e-01 8.92105287e-02]
 [3.24341189e-03 9.96756588e-01]
 [9.98867517e-01 1.13248306e-03]
 [1.00727228e-03 9.98992728e-01]
 [9.02980248e-01 9.70197522e-02]
 [1.00000000e+00 5.25489414e-13]
 [2.33991402e-04 9.99766009e-01]
 [5.49013655e-02 9.45098634e-01]
 [3.43706371e-03 9.96562936e-01]
 [9.98946266e-01 1.05373359e-03]]
DecisionTreeClassifier 개별 정확도 : 0.8811188811188811
DecisionTreeCla

c:\Users\Playdata\anaconda3\envs\ml_env\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
